# DBSCAN

Density-Based Spatial Clustering of Applications with Noise or DBSCAN for short is an algorithm that groups data points together that are in areas of high density and then identify data in lower density regions as noise. 
Similar to the purely hierarchical clustering method discussed previously, this makes no assumption about the shape of the clusters. 
Unlike *k*-means and GMMs, which assume spheres and ellipses, respectively. 
Additionally, it does not assume some given cluster number. 

To discuss the DBSCAN algorithm, let's look at a slightly different dataset. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons, make_blobs

X1, y1 = make_moons(n_samples=200, noise=0.075, random_state=42)
X2, y2 = make_blobs(n_samples=200, centers=2, random_state=42) 
X2 /= np.array([10, 10])
X2 += np.array([1, 1])

X = np.vstack([X1, X2])
y = np.hstack([y1, y2])

fig, ax = plt.subplots(figsize=(4, 4))
ax.plot(*X.T, '.')
ax.axis('equal')
plt.show()

We see that there are four cluster, two of which are intersceting moons and two are circular. 
First, we can see how GMMs would cope. 

In [ ]:
from sklearn.mixture import GaussianMixture

gmm = GaussianMixture(n_components=4, random_state=42)
gmm.fit(X)
label = gmm.predict(X)
prob = gmm.predict_proba(X)
size = 50 * prob.max(1) ** 2

fig, ax = plt.subplots(figsize=(4, 4))
ax.scatter(*X.T, s=size, c=label)
ax.axis('equal')
plt.show()

It is clear, that even though the Gaussian mixture model is told that there are 4 clusters, it fails to find the moons, given they cannot be described with an ellipitical model. 
However, can the DBSCAN approach improve on this. 

## DBSCAN Algorithm

At the start of the algorithm, all of the data points are classified as *unvisited*. 
Additionally, we will need to know the distance between all the points. 
So let's generate that in the same fashion as previously. 

In [3]:
from scipy.spatial.distance import pdist, squareform

visited = [False] * X.shape[0]

distances = squareform(pdist(X))
np.fill_diagonal(distances, np.inf)

We will then visit the first data point, and update the visited status. 

In [4]:
visited[0] = True

For that first data point, we want to find all of the other points that are less then some distance `eps` from it, these are the neighbours of the data point. 
This `eps` is one of the hyperparameters of the DBSCAN algorithm, for this example, we will use a value of 0.1.

In [ ]:
eps = 0.1
np.where(distances[0] < eps)

````{margin}
```{note}
This classification an noise may change in the future. 
```
````
If the data point has fewer than `min_samples` (the second hyperparameter, here we use 5), then the point is identified a noise. 
We will do this by making the value of a `labels` array -1. 

In [16]:
min_samples = 5
labels = np.zeros(X.shape[0], dtype=int)

if (distances[0] < eps).sum() < min_samples:
    labels[0] = -1

However, if the point has more than `min_samples` neighbours, we classify this as a core point and begin to expand the cluster around it. 
Data point 338 has many neighbours. 

In [ ]:
(distances[338] < eps).sum()

A core point is assigned a `cluster_id`, which starts from 0. 

In [18]:
cluster_id = 0

labels[338] = cluster_id

And then for each point in the neighbour of a data point, we perform the following: 
- If the point is unvisited, its neighbours are found and marked as visited and if it has more then the `min_samples` of neighbours then these are included as neighbours of the core point. 
- If the point has been visited and it currently defined as noise, the label is changed to that of the core point.

This is shown for data point 338 below. 

In [19]:
neighbours = np.where(distances[338] < eps)[0]

j = 0
while j < neighbours.size:
    current = neighbours[j]
    if not visited[current]:
        visited[current] = True
        new_neighbors = np.where(distances[current] <= eps)[0]
        if new_neighbors.size >= min_samples:
            neighbours = np.concatenate([neighbours, new_neighbors])
    if labels[current] == -1:
        labels[current] = cluster_id
    j += 1
cluster_id += 1

We can see that this has lead to a large number of particles being visited. 

In [ ]:
np.sum(visited)

To put the algorithm together, we will reset some of the bookkeeping. 

In [21]:
visited = [False] * X.shape[0]
labels = np.zeros(X.shape[0], dtype=int) - 1
cluster_id = 0

for i in range(X.shape[0]):
    if not visited[i]:
        visited[i] = True
        neighbours = np.where(distances[i] <= eps)[0]
        if neighbours.size < min_samples:
            labels[i] = -1
        else:
            labels[i] = cluster_id
            j = 0
            while j < neighbours.size:
                current = neighbours[j]
                if not visited[current]:
                    visited[current] = True
                    new_neighbors = np.where(distances[current] <= eps)[0]
                    if new_neighbors.size >= min_samples:
                        neighbours = np.concatenate([neighbours, new_neighbors])
                if labels[current] == -1:
                    labels[current] = cluster_id
                j += 1
            cluster_id += 1

Using these values of the hyperparameters, the DBSCAN algorithm has produced 12 clusters. 

In [ ]:
np.unique(labels)

And identified 134 of the 400 data points as noise. 

In [ ]:
(labels == -1).sum()

We try to visualise this but there probably isn't enough colour fidelity in the `matplotlib` standard plotting. 

In [ ]:
fig, ax = plt.subplots(figsize=(4, 4))
ax.scatter(*X.T, c=labels)
ax.axis('equal')

## HDBSCAN

Before we leave clustering behind, we will look at one final approach, Hierarchical DBSCAN{cite}`Campello2013`. 
HDBSCAN brings the hierarchical clustering approach together with DBSCAN.
One of the major benefits of HDBSCAN over DBSCAN is that it is not necessary to define the hyperparameters. 

In [ ]:
from sklearn.cluster import HDBSCAN

hdbscan = HDBSCAN()
label = hdbscan.fit_predict(X)

fig, ax = plt.subplots(figsize=(4, 4))
ax.scatter(*X.T, c=label)
ax.axis('equal')

A naïve usage of the `scipy.cluster.HBDSCAN` method is able to perfectly capture the data.

This isn't to say that HDBSCAN is perfect, and there is still a place for the other clustering methods.
Not least due to the high computational cost of HDBSCAN compared to the more straight-forward approaches. 